<a href="https://colab.research.google.com/github/eiglesiasr/NoHelmetNet/blob/main/src/Full_birmania_dataset_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://token@github.com/eiglesiasr/NoHelmetNet.git

Cloning into 'NoHelmetNet'...
remote: Enumerating objects: 559, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 559 (delta 60), reused 86 (delta 30), pack-reused 426 (from 1)
Receiving objects: 100% (559/559), 60.22 MiB | 14.80 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [16]:
import pandas as pd
import os
import shutil
import json
from sklearn.model_selection import train_test_split

seed = 42

In [3]:

#cargando json con el encoding de labels a un diccionario
ruta_json='NoHelmetNet/dataset/labels.json'
with open(ruta_json, 'r') as f:
      labels_dict = json.load(f)

In [21]:
# leyendo todos los csv con las anotaciones de las predicciones en un dataframe
carpeta_csv = 'NoHelmetNet/raw/Birmania/'

lista_dfs = []

for archivo in os.listdir(carpeta_csv):
    if archivo.endswith('.csv'):
        ruta_archivo = os.path.join(carpeta_csv, archivo)
        df = pd.read_csv(ruta_archivo)
        nombre_base = os.path.splitext(archivo)[0]
        df['nombre_carpeta'] = nombre_base
        df['frame'] = df['frame_id'].apply(lambda x: f'{x:02d}.jpg')
        lista_dfs.append(df)

# Concatena todos los DataFrames en uno solo
df_final = pd.concat(lista_dfs, ignore_index=True)
df_final.head()

,track_id,frame_id,x,y,w,h,label,nombre_carpeta,frame
0,_xfiat5odq,1,265,346,43,82,DHelmet,Mandalay_1_1,01.jpg
1,_xfiat5odq,2,258,346,42,80,DHelmet,Mandalay_1_1,02.jpg
2,_xfiat5odq,3,250,345,42,80,DHelmet,Mandalay_1_1,03.jpg
3,_xfiat5odq,4,242,344,42,79,DHelmet,Mandalay_1_1,04.jpg
4,_xfiat5odq,5,235,344,41,78,DHelmet,Mandalay_1_1,05.jpg


In [18]:
# generando un dataframe que contenga un registro unico por imagen para
# particionar en train y test y luego parsear a formato yolov8
df_imagenes_unicas = df_final[['nombre_carpeta', 'frame']].drop_duplicates().reset_index(drop=True)
print(f"Imagenes unicas: {len(df_imagenes_unicas)}")

indices_imagenes = df_imagenes_unicas.index

train_indices, test_indices = train_test_split(
    indices_imagenes,
    test_size=0.3,
    random_state=seed
)

df_imagenes_train = df_imagenes_unicas.loc[train_indices].reset_index(drop=True)
df_imagenes_test = df_imagenes_unicas.loc[test_indices].reset_index(drop=True)

print(f"Número de imágenes para entrenamiento: {len(df_imagenes_train)}")
print(f"Número de imágenes para prueba: {len(df_imagenes_test)}")


Imagenes unicas: 200
Número de imágenes para entrenamiento: 140
Número de imágenes para prueba: 60


In [23]:
# funcion que dado un df de imagenes y carpeta genera tanto las imagenes como
# anotaciones en formato yolov8 en la carpeta dataset:

def create_dataset_yolo_from_raw(ruta_origen,ruta_destino_imagenes,ruta_destino_anotaciones,df_imagenes,df_anotaciones,image_width,image_height,labels_dict):
  for index, row in df_imagenes.iterrows():
    #copiando imagen
    nombre_carpeta = row['nombre_carpeta']
    frame_jpg = row['frame']
    frame = int(row['frame'].split('.')[0])
    ruta_imagen_origen = os.path.join(ruta_origen, nombre_carpeta, frame_jpg)
    ruta_imagen_destino = os.path.join(ruta_destino_imagenes, f'{nombre_carpeta.lower()}_{frame:03d}.jpg')
    shutil.copy(ruta_imagen_origen, ruta_imagen_destino)

    #generando anotaciones
    txt_filename = f"{nombre_carpeta.lower()}_{frame:03d}.txt"
    txt_filepath = os.path.join(ruta_destino_anotaciones, txt_filename)
    for index, row in df_anotaciones[(df_anotaciones.nombre_carpeta==nombre_carpeta) & (df_anotaciones.frame==frame_jpg)].iterrows():
      class_id=labels_dict[row['label']]
      x_center = int(row['x']+row['w']/2)
      y_center = int(row['y']+row['h']/2)

      x_center_norm = x_center / image_width
      y_center_norm = y_center / image_height
      width_norm = row['w'] / image_width
      height_norm = row['h'] / image_height

      yolov8_annotation=f"{class_id} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}\n"

      with open(txt_filepath, 'a') as f:
          f.write(yolov8_annotation)


In [24]:
#eliminando dataset de train y de test si existieran
!rm -rf NoHelmetNet/dataset/images/train/*
!rm -rf NoHelmetNet/dataset/labels/train/*
!rm -rf NoHelmetNet/dataset/images/val/*
!rm -rf NoHelmetNet/dataset/labels/val/*

In [26]:
#generando dataset de entrenamiento
create_dataset_yolo_from_raw(
    ruta_origen='NoHelmetNet/raw/Birmania',
    ruta_destino_imagenes='NoHelmetNet/dataset/images/train',
    ruta_destino_anotaciones='NoHelmetNet/dataset/labels/train',
    df_imagenes=df_imagenes_train,
    df_anotaciones=df_final,
    image_width=1920,
    image_height=1080,
    labels_dict=labels_dict,
)

In [27]:
#generando dataset de entrenamiento
create_dataset_yolo_from_raw(
    ruta_origen='NoHelmetNet/raw/Birmania',
    ruta_destino_imagenes='NoHelmetNet/dataset/images/val',
    ruta_destino_anotaciones='NoHelmetNet/dataset/labels/val',
    df_imagenes=df_imagenes_test,
    df_anotaciones=df_final,
    image_width=1920,
    image_height=1080,
    labels_dict=labels_dict,
)